# Datalab Quick Starter
Simple notebook where you only need to implement the prediction logic.
<br>
The first code cell is responsable for importing Python dependecies and setting up credentials. 
<br>
Change the _bucket_ value to your bucket and run the cell (_Ctrl+Enter_).
<br>
You should run a cell (_Ctrl+Enter_) every time you change something on it.

In [3]:
import pandas as pd
import numpy as np
from google.cloud import storage

project_id = 'paul-the-octopus-hackaton'
bucket = 'ciandt_octopus_wender__dot__zacariasxavier__at__gmail__dot__com'

## Load Data from BigQuery
Six BigQuery tables are loaded into Pandas DataFrames and can be used in your prediction algorithm.
<br>
You can run (_Ctrl+Enter_) all the cells or only what you'll use for the prediction.

**fifa_rank**: The FIFA rank for national teams.

In [4]:
fifa_rank = pd.io.gbq.read_gbq('SELECT * FROM [paul_the_octopus_dataset.fifa_rank]', project_id=project_id, verbose=False)
fifa_rank.head(2)

,Rank,Team_Code,Team,Total_Points
0,1,GER,Germany,1609
1,2,BRA,Brazil,1489


**matches_schedule**: The list of matches for the 2018 World Cup.

In [5]:
matches_schedule = pd.io.gbq.read_gbq('SELECT date, TRIM(home) as home, TRIM(away) as away FROM `paul_the_octopus_dataset.matches`', project_id=project_id, verbose=False, dialect='standard')
matches_schedule.head(2)

,date,home,away
0,26 June 2018 17:00 MSK (UTC+3),Australia,Peru
1,21 June 2018 20:00 YEKT (UTC+5),France,Peru


**matches_history**: First phase match results since 1930 World Cup.

In [6]:
matches_history = pd.io.gbq.read_gbq('SELECT * FROM [paul_the_octopus_dataset.matches_history]', project_id=project_id, verbose=False)
matches_history.head(2)

,year,home,home_score,away_score,away
0,1998,Yugoslavia,1,0,Iran
1,1998,Germany,2,0,Iran


**panini_players**: List of players contained in the Panini stickers album.

In [7]:
panini_players = pd.io.gbq.read_gbq('SELECT * FROM [paul_the_octopus_dataset.panini_players]', project_id=project_id, verbose=False)
panini_players.head(2)

,country,name
0,Peru,Pedro Gallese
1,Peru,Carlos Cáceda


**sofifa_players_2018**: Player statistics (not limited to the participating teams, so some filtering is needed).

In [8]:
sofifa_players_2018 = pd.io.gbq.read_gbq('SELECT * FROM [paul_the_octopus_dataset.sofifa_players_2018]', project_id=project_id, verbose=False)
sofifa_players_2018.head(2)

,Rank,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,...,Reactions,Short_passing,Shot_power,Sliding_tackle,Sprint_speed,Stamina,Standing_tackle,Strength,Vision,Volleys
0,16707,M. Rafique,24,India,55,60,None,€0,€0,1535,...,48,63,52,52,67,61,48,42,48,47
1,17553,S. Brotherton,20,New Zealand,52,65,None,€0,€0,1177,...,37,36,33,55,62,51,53,74,28,24


**teams**: List of teams with some historical data.

In [9]:
teams = pd.io.gbq.read_gbq('SELECT * FROM [paul_the_octopus_dataset.teams]', project_id=project_id, verbose=False)
teams.head(2)

,Team,Previous__appearances,Previous__titles,Previous__finals,Previous__semifinals
0,Iceland,0,0,0,0
1,Panama,0,0,0,0


## Prediction Logic
 The next cell should contain a function called _predict_. It receives a dataframe of the world cup matches ("home" and "away" columns) and should return another dataframe with the scores ("home", "home_score", "away_score", "away" columns).
 <br>
 There is an example of function implementation below, where the better ranked team scores 2 x 0.
 <br>
 <font color="red">**Don't forget to run the cell (_Ctrl+Enter_) after you finish coding.**</font>

In [3]:
def predict(matches):
  
    # Initializing weights 
    host_weight = 20.0
    value_weight = 2.1
    age_weight = 0.3
    teamRankFifa = 0.1
    predictResult = []
    
    # Creating DataFrame
    overall = pd.DataFrame(0.0, index= set(matches_schedule["home"].values) , columns=["age", "value", "history" , "host", "previous_match", "rank"])
    
    # Initializing Host weight as 0.0 for every city, and 20.0 for host-City ( 2018-Russia )
    overall["host"] = 0.0
    overall.loc["Russia"]["host"] = host_weight
    
    # Create Overall DataFrame with necessary information
    # Loop through all World Cup 2018 countries, for each country
    # Collect Overall and Age MEAN values for the best 23 players of each country 
    # Collect Rank information of each country
    # Fill missing values of Age and Value as the minimum value of the column
    for country in overall.index:
      overall.loc[country]["value"] = sofifa_players_2018[sofifa_players_2018["Nationality"] == country].sort_values("Overall", ascending=False)[:23]["Overall"].mean()
      overall.loc[country]["age"] = sofifa_players_2018[sofifa_players_2018["Nationality"] == country].sort_values("Overall", ascending=False)[:23]["Age"].mean()
      overall.loc[country]["rank"] = fifa_rank[fifa_rank["Team"] == country]["Rank"]
    overall["age"].fillna(overall["age"].min(), inplace=True)
    overall["value"].fillna(overall["value"].min(), inplace=True)

    # Loop through the items of Overall Dataframe
    # Store home team information to the variable HOME, and away information to the variable AWAY
    for index, match in matches_schedule.sort_values("date").iterrows():
      home = overall.loc[match["home"]]
      away = overall.loc[match["away"]]
      
      # Score Calc: Calculate the difference between value for the home and away teams (w=Weight)
      # wPreviousMatch = Start with 0, and then update considering if the team won or lost
      # home (wAge + wValue + wHost + wPreviousMatch - wFifaRank) - away (wAge + wValue + wHost + wPreviousMatch + wFifaRank)
      
      score = (age_weight*home["age"] + value_weight*home["value"] + home["host"] + home["previous_match"] - teamRankFifa*home["rank"] ) - (age_weight*away["age"] + value_weight*away["value"] + away["previous_match"] + away["host"] - teamRankFifa*home["rank"])


      # Check the difference between teams are higher than 10 or lower than -10
      # How many goals the winning team will score
      # Test to see if the other team is good enough to score ( 0.3 + Winner(wValue) - Looser(wValue) / 10 )
      # Check to see if the number of goals is too big or higher than the winning team and correct value

      if score > 10:
        home_score = int(score/10 + .9)
        overall.loc[match["away"]]["previous_match"] =  home_score + 1.5
        # Test to see is the other team scores
        away_score_test = int(0.3 + abs(home["value"]-away["value"])/10)
        if ( away_score_test < int(score/10)):
              away_score = away_score_test
        elif ( int(score)/10 == away_score_test and away_score_test-1 > 0 ):
              away_score = away_score_test-1
        else: 
          away_score = 0
      elif score < -10:
        away_score = int(-score/10 + .9)
        overall.loc[match["home"]]["previous_match"] = away_score + 1.5
        home_score_test = int(0.3 + abs(away["value"]-home["value"])/10)
        if ( home_score_test < int(score/10)):
              home_score = home_score_test
        elif ( int(score)/10 == home_score_test and home_score_test-1 > 0 ):
              home_score = home_score_test-1
        else: 
          home_score = 0
      
      # If the Teams are similar and will be draw
      # Calculate the number of goals scored based on the effectiviness of the teams
      
      else:
        if(home["rank"] + away["rank"] > 25 ):
          draw_score = int(0.8 + abs(away["value"]-home["value"])/10)
        else:
          draw_score = int( 1.2 + ((home["rank"]+away["rank"]/10) + abs(away["value"]-home["value"])/10)/2)
        home_score = draw_score
        away_score = draw_score
          

      # Append each result considering team names and score to the predictResult array
      # return as DataFrame containing all games and results ordered.
      
      predictResult.append([match["home"], home_score, away_score ,match["away"]])
    return (pd.DataFrame(predictResult, columns=["home", "home_score", "away_score", "away"]))

## Prediction Test
Executing the next cell will test your prediction code against the four last World Cups. This is useful to verify if a code change makes a prediction better or worse.

In [28]:
def winner_points(row, winner_team, loser_team):
  winner_score = '%s_score' % winner_team
  loser_score = '%s_score' % loser_team
  expected_winner_score = 'expected_%s_score' % winner_team
  expected_loser_score = 'expected_%s_score' % loser_team

  # winner score
  if row[expected_winner_score] == row[winner_score]:
    return 18
  # winner/loser difference
  if row[expected_winner_score] - row[expected_loser_score] == row[winner_score] - row[loser_score]:
    return 15
  # loser score
  elif row[expected_loser_score] == row[loser_score]:
    return 12
  return 10
  
def score_points(expected, prediction):
  points = 0
  
  for i, row in expected.merge(prediction, left_on=['home','away'], right_on=['home','away'], how='left').iterrows():
    
    # exact score
    if (row['expected_home_score'], row['expected_away_score']) == (row['home_score'], row['away_score']):
      points += 25
    
    # home won
    elif row['expected_home_score'] > row['expected_away_score'] and row['home_score'] > row['away_score']:
      points += winner_points(row, 'home', 'away')
      
    # away won
    elif row['expected_away_score'] > row['expected_home_score'] and row['away_score'] > row['home_score']:
      points += winner_points(row, 'away', 'home')
      
    # draw
    elif row['home_score'] == row['away_score']:
      if row['expected_away_score'] == row['expected_home_score']:
        points += 15
      else:
        points += 4
        
  return points

def select_past_matches(year):
  m = matches_history[matches_history['year'] == year][['home','away', 'home_score', 'away_score']]
  m.columns.values[[2, 3]] = ['expected_home_score', 'expected_away_score']
  return m

def test(year):
  old_scores = select_past_matches(year)
  prediction = predict(old_scores[['home','away']])
  return score_points(old_scores, prediction)

test_results = pd.DataFrame({'Year': [2002, 2006, 2010, 2014]})
test_results['Points'] = test_results['Year'].apply(test)
test_results['%'] = test_results['Points'] * 100 / 48 / 25 
print 'Test for old World Cups:'
test_results

Test for old World Cups:


,Year,Points,%
0,2002,0,0.00
1,2006,0,0.00
2,2010,0,0.00
3,2014,15,1.25


## Predicting and Uploading to GCS
Next cell executes your prediction function for the 2018 World Cup and uploads a CSV file to your GCS bucket.
<br>
<font color="red">**Don't forget to give the GCE instance (where Datalab is running) permission to write into the bucket.**</font> Instructions __[here](https://docs.google.com/document/d/1SJRglulgz2x63h24wD3SgBZOA1DgdTtTI92Z66wc6oM/edit)__.

In [21]:
csv = predict(matches_schedule[['home','away']]).to_csv(index=False)

storage_client = storage.Client(project=project_id)
bucket = storage_client.get_bucket(bucket)
blob = bucket.blob('predictions.csv')
blob.content_type = 'text/plain'
blob.upload_from_string(csv)

Forbidden: 403 GET https://www.googleapis.com/storage/v1/b/ciandt_octopus_wender__dot__zacariasxavier__at__gmail__dot__com?projection=noAcl: 1074176232054-compute@developer.gserviceaccount.com does not have storage.buckets.get access to ciandt_octopus_wender__dot__zacariasxavier__at__gmail__dot__com.

In [20]:
predict(matches_schedule[['home','away']]).to_csv("predictions.csv", index=False)